## Finding and Cleaning r/loseit Challenge Data

This is done using the reddit API praw and the Google Sheets API gspread. Add more info ...

In [1]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import praw
import datetime as dt
import markdown
from lxml import etree
import lxml
import gspread
import re
import matplotlib.pyplot as plt
import seaborn as sns
from oauth2client.service_account import ServiceAccountCredentials
from pathlib import Path
import os

This is a function used to sort some of the lists in human ordered form.

In [2]:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    '''
    return [ atoi(c) for c in re.split('(\d+)', text) ]

Create the Data Directories

In [3]:
if not os.path.exists('./data/raw_data/'):
    os.makedirs('./data/raw_data/')
if not os.path.exists('./figures/'):
    os.makedirs('./figures/')
if not os.path.exists('./data/cleaned/'):
    os.makedirs('./data/cleaned/')

In [4]:
reddit = praw.Reddit('loseit_data')

In [5]:
loseit_sub = reddit.subreddit('loseit')
challenge_posts = loseit_sub.search("loseit challenge tracker", limit=1000)
topics_dict = { "title":[], "score":[], "id":[], "url":[], "comms_num": [],
                "created": [], "body":[]}

In [6]:
for submission in challenge_posts:
    topics_dict["title"].append(submission.title)
    topics_dict["score"].append(submission.score)
    topics_dict["id"].append(submission.id)
    topics_dict["url"].append(submission.url)
    topics_dict["comms_num"].append(submission.num_comments)
    topics_dict["created"].append(submission.created)
    topics_dict["body"].append(submission.selftext)

In [7]:
topics_data = pd.DataFrame(topics_dict)

In [8]:
def get_date(created):
    return dt.datetime.fromtimestamp(created)
_timestamp = topics_data["created"].apply(get_date)
topics_data = topics_data.assign(timestamp = _timestamp)

In [9]:
topics_data.to_csv('loseit_search_history.csv')

Now that we have searched through old loseit posts, we need to find the urls.

In [10]:
links = []
for body in topics_dict['body']:
    try:
        doc = etree.fromstring(markdown.markdown(re.sub('[\\n]', '', body)))
        for link in doc.xpath('//a'):
            web_url = link.get('href')
            if bool(re.search('spreadsheet',web_url)):
                links.append(web_url)
    except etree.XMLSyntaxError:
        print('',end='\t')

In [11]:
unique_spreadsheets = list(set(links))

We now use the Google API to download the data.

In [12]:
# use creds to create a client to interact with the Google Drive API
names = []
for spreadsheet_link in unique_spreadsheets:
    scope = ['https://spreadsheets.google.com/feeds']
    creds = ServiceAccountCredentials.from_json_keyfile_name('loseit-sheets-6012c29a1f40.json', scope)
    gc = gspread.authorize(creds)
    sht = gc.open_by_url(spreadsheet_link)
    if bool(re.search('nter', sht.title)) == False and bool(re.search('/r/', sht.title)) == False and bool(re.search('Calculator', sht.title)) == False:
        sheet_name = re.sub('_\(responses\)' ,'', re.sub(',', '', re.sub('\]','', re.sub('\[','', re.sub(' ', '_', re.sub('  ', '_', re.sub('-', '', sht.title.lower())))))))
        if sheet_name not in names:
            print(f'sheet name: {sheet_name}, link: {spreadsheet_link}')
            names.append(sheet_name)
            try: 
                data_sheet = sht.worksheet('Tracker')
                data_vals = data_sheet.get_all_values()
                data_df = pd.DataFrame(data_vals[1:-2], columns=data_vals[0])
                data_df.to_csv('./data/raw_data/' + sheet_name + '.csv')
            except gspread.WorksheetNotFound:
                try:
                    data_sheet = sht.worksheet('Master Spreadsheet')
                    data_vals = data_sheet.get_all_values()
                    data_df = pd.DataFrame(data_vals[1:-2], columns=data_vals[0])
                    data_df.to_csv('./data/raw_data/' + sheet_name + '.csv')
                except gspread.WorksheetNotFound:
                    print('',end='\t')#sheet_name)
            else:
                print('',end='\t')#sheet_name)

sheet name: rebirth_challenge_2017, link: https://docs.google.com/spreadsheets/d/1BAsiKlOCty_2sZeb93ogQ40tNJSIgREfar5NcnFGU4E/edit?usp=sharing
	sheet name: spring_into_summer_challenge, link: https://docs.google.com/spreadsheets/d/11-26-l8mHoBq-XMUTtq7AlXfIWvPllBSL1r1hbq1xnA/edit#gid=1429543432
sheet name: spring_time_to_energize_challenge, link: https://docs.google.com/spreadsheets/d/1R62EK0PZeQJL0Cfd1UyMThaAU8IDZgD_f7R9aqonGVU/edit#gid=1340513170
	sheet name: new_year_new_you_2016_challenge_master_spreadsheet, link: https://docs.google.com/spreadsheets/d/1PQpubmKrJcrA_bEBJZD_YUo87WYkxiu09CFK-ppehlQ/edit#gid=1711453384
	sheet name: lord_of_the_rings_loseit_summer_challenge_sign_ups, link: https://docs.google.com/spreadsheets/d/1VIRc3Xu7havYhzHWYIyt_gWacTUP3T33NGCIG-lXBfw/edit?usp=sharing
	sheet name: tracker_loseit_spring_semester_challenge, link: https://docs.google.com/spreadsheet/ccc?key=0Ajn9WUix2Lq1dFpHWG14N0hOV1Q2SzUxRXlBNE9JUEE&usp=drive_web#gid=0
	sheet name: super_hero_summer

The next step is cleaning up some of the column information, and removing the information that is not useful for analysis.

In [23]:
weeks_col = {f'W{x}':f'Week {x}' for x in range(0, 11)}
new_names = {'W0 (SW)': 'Week 0', 'Sex': 'Gender', 'Male, Female, Other': 'Gender',
             'TEAM': 'Team', 'Teams': 'Team', 'Challenge GW': 'Challenge Goal Weight',
             'Challenge SW': 'Week 0', 'MyFitnessPal Username/Link': 'MFP'}

In [30]:
df_list = []
p = Path('./data/raw_data/')
for challenge in p.rglob('*.csv'):
    # Read in the csv files and change some of the column names
    test_df = pd.read_csv(challenge, index_col=0)
    test_df.dropna(axis=1, how='all')
    test_df.columns = test_df.columns.str.strip().str.replace('?', '').str.replace(':','')
    test_df.rename(columns=new_names, inplace=True)
    
    # timestamp
    if 'Timestamp' not in test_df:
        test_df['Timestamp'] = np.NaN
    # Age
    test_df['Age'] = test_df[test_df.filter(regex=re.compile('Age', re.IGNORECASE)).columns[0]]
    
    # Gender
    if len(test_df.filter(regex=re.compile('Sex', re.IGNORECASE)).columns):
        test_df['Gender'] = test_df[test_df.filter(regex=re.compile('Sex', re.IGNORECASE)).columns[0]] 
    if len(test_df.filter(regex=re.compile('Gender', re.IGNORECASE)).columns):
        test_df['Gender'] = test_df[test_df.filter(regex=re.compile('Gender', re.IGNORECASE)).columns[0]] 
    if 'Gender' not in test_df:
        test_df['Gender'] = 'Unknown'

    # Ignore KGS
    if len(test_df.filter(regex=re.compile('kgs', re.IGNORECASE)).columns):
        test_df.drop(test_df.filter(regex=re.compile('kgs', re.IGNORECASE)).columns[0], axis=1, inplace=True)
    
    # Keep Just Starting BMI
    test_df.drop(test_df.filter(regex=re.compile('BMI', re.IGNORECASE)).columns[1:], axis=1, inplace=True)
    
    # Username
    test_df.columns = test_df.columns.str.replace(test_df.filter(like='name').columns[0], 'Username')
    
    # Weigh-in Data
    test_df.rename(columns=weeks_col, inplace=True)            
    if len(test_df.filter(regex=re.compile("week 0", re.IGNORECASE)).columns):
        test_df['Week 0'] = test_df[test_df.filter(regex=re.compile("week 0", re.IGNORECASE)).columns[0]]
    elif len(test_df.filter(regex=re.compile("sign-up", re.IGNORECASE)).columns):
        test_df['Week 0'] = test_df[test_df.filter(regex=re.compile("sign-up", re.IGNORECASE)).columns[0]]
    elif len(test_df.filter(regex=re.compile("start weight", re.IGNORECASE)).columns):
        test_df['Week 0'] = test_df[test_df.filter(regex=re.compile("start weight", re.IGNORECASE)).columns[0]]
    elif len(test_df.filter(regex=re.compile("Signup weight", re.IGNORECASE)).columns):
        test_df['Week 0'] = test_df[test_df.filter(regex=re.compile("Signup weight", re.IGNORECASE)).columns[0]]
    elif len(test_df.filter(regex=re.compile('What is your current weight', re.IGNORECASE)).columns):
        test_df['Week 0'] = test_df[test_df.filter(regex=re.compile('What is your current weight', re.IGNORECASE)).columns[0]]

    # Height
    test_df['Height'] = test_df[test_df.filter(regex=re.compile("Height", re.IGNORECASE)).columns[0]]   

    # Highest Weight
    if len(test_df.filter(regex=re.compile('Highest', re.IGNORECASE)).columns):
        test_df['Highest Weight'] = test_df[test_df.filter(regex=re.compile('Highest', re.IGNORECASE)).columns[0]]
    else:
        test_df['Highest Weight'] = np.NaN
    
    # Has NSV
    test_df['Has NSV'] = test_df[test_df.filter(regex=re.compile("NSV", re.IGNORECASE)).columns[0]].notnull().astype('int')
    test_df['NSV Text'] = test_df[test_df.filter(regex=re.compile("NSV", re.IGNORECASE)).columns[0]].astype(str).replace('nan', '')
    
    # Goal Weight
    test_df['Challenge Goal Weight'] = test_df[test_df.filter(regex=re.compile("Goal Weight", re.IGNORECASE)).columns[0]]

    # Has a food tracker
    if len(test_df.filter(regex=re.compile('MyFitnessPal', re.IGNORECASE)).columns):
        test_df['MFP'] = test_df[test_df.filter(regex=re.compile('MyFitnessPal', re.IGNORECASE)).columns[0]].notnull().astype('int')    
    test_df['Has MFP'] = test_df[test_df.filter(regex=re.compile("MFP", re.IGNORECASE)).columns[0]].notnull().astype('int')
    if len(test_df.filter(regex=re.compile("Loseit", re.IGNORECASE)).columns):
        test_df['Has Loseit'] = test_df[test_df.filter(regex=re.compile("Loseit", re.IGNORECASE)).columns[0]].notnull().astype('int')
    else:
        test_df['Has Loseit'] = 0
    test_df['Has Food Tracker'] = test_df['Has MFP'] + test_df['Has Loseit']
    test_df['Has Food Tracker'] = test_df['Has Food Tracker'].replace(2,1)
    
    # fitness tracker
    if len(test_df.filter(regex=re.compile('Fitbit', re.IGNORECASE)).columns):
        test_df['Has Activity Tracker'] = test_df[test_df.filter(regex=re.compile('Fitbit', re.IGNORECASE)).columns[0]].notnull().astype('int')
    elif len(test_df.filter(regex=re.compile('Fitness tracker', re.IGNORECASE)).columns):
        test_df['Has Activity Tracker'] = test_df[test_df.filter(regex=re.compile('Fitness Tracker', re.IGNORECASE)).columns[0]].notnull().astype('int')
    elif len(test_df.filter(regex=re.compile('Garmin', re.IGNORECASE)).columns):
        test_df['Has Activity Tracker'] = test_df[test_df.filter(regex=re.compile('Garmin', re.IGNORECASE)).columns[0]].notnull().astype('int')
    elif len(test_df.filter(regex=re.compile('Strava', re.IGNORECASE)).columns):
        test_df['Has Activity Tracker'] = test_df[test_df.filter(regex=re.compile('Strava', re.IGNORECASE)).columns[0]].notnull().astype('int')
 
    # Starting Weight
    test_df['Starting Weight'] = test_df['Week 0']
        
    # Create the final Data Frame
    col_weeks = test_df.filter(regex=re.compile('Week', re.IGNORECASE)).columns.tolist()
    col_weeks.sort(key=natural_keys)
    col_names = ['Timestamp', 'Username', 'Team', 'Age', 'Gender', 'Height','Highest Weight', 
                 'Starting Weight', 'Challenge Goal Weight', 'Starting BMI', 'Has NSV', 
                 'Has Food Tracker', 'Has Activity Tracker', 'NSV Text']
    
    data_cols = col_names + list(col_weeks)
    data_df = test_df[data_cols]

    df_list.append((challenge.stem, data_df))

Now that the data contains only what we are interested in learning, we need to fill in any missing values before we combine all of the challenges together.

In [31]:
big_df_list = []

for data in df_list:
    df = data[1].copy()

    # Some odties in the data
    if data[0] == 'spring_time_to_energize_challenge':
        df.drop([448, 828], inplace=True)
        df.replace({'ERROR': np.NaN}, inplace=True)
    if data[0] == 'autumn_animal_challenge':
        df.drop(971, inplace=True)
        df.replace({'#DIV/0!': np.NaN, 'old':np.NaN}, inplace=True)
    if data[0] == 'rebirth_challenge_2017':
        df.drop(['Week 7', 'Week 8'], axis=1, inplace=True)
        df.replace({'20s': 25, 'Yes': np.NaN}, inplace=True)


    df.dropna(subset=['Username', 'Challenge Goal Weight'], axis=0, inplace=True)
    df.loc[pd.isnull(df['Gender']), 'Gender'] = 'Unknown'
    df.loc[~df['Gender'].isin(['Female', 'Male', 'Unknown']), 'Gender'] = 'Other'
    df.loc[pd.isnull(df['Highest Weight']), 'Highest Weight'] = df['Week 0']
    df['Timestamp'] = df['Timestamp'].fillna(axis=0, method='ffill', limit=10)

    '''
    Now we want to convert the series into the correct types
    '''
    numberic = ['Age', 'Height','Highest Weight', 'Starting Weight', 'Challenge Goal Weight', 'Starting BMI']
    df[numberic] = df[numberic].astype(np.float64)
    
    '''
    Now we need to work on removing those who dropped out of the challenge. 
    First, if only one weigh-in was missed we will fill it with the previous weeks 
    weigh-in. Next, we remove any that are missing the final weigh-in, and lastly, 
    we fill any of the remaining missing values with the previous weeks data.
    '''
    
    weight_cols = df.columns.values[14:].tolist()

    df[weight_cols] = df[weight_cols].fillna(axis=1, method='ffill', limit=1)
    df.dropna(axis=0, subset=[weight_cols[-1]], inplace=True)
    df[weight_cols] = df[weight_cols].fillna(axis=1, method='ffill').astype(np.float64)

    '''
    Now we want to create a couple more columns for total weight lost, 
    percentage lost, percentage of challenge weight lost
    '''
    
    new_cols = ['Final Weight', 'Total Challenge Loss', 'Challenge Percentage Lost', 'Percent of Challenge Goal']
    
    df['Challenge Goal Loss'] = df['Starting Weight'].astype(np.float64) - df['Challenge Goal Weight'].astype(np.float64)
    df[new_cols[0]] = df[weight_cols[-1]]
    df[new_cols[1]] = df[weight_cols[0]] - df[weight_cols[-1]]
    df[new_cols[2]] = (df[new_cols[1]] / df[weight_cols[0]]) * 100
    df[new_cols[3]] = (df[new_cols[1]] / (df['Starting Weight'].astype(np.float64) - df['Challenge Goal Weight'].astype(np.float64))).replace(np.inf,0).replace(-np.inf,0) * 100
    df[new_cols] = df[new_cols].astype(np.float64)
    
    df = df[df.columns.values[:14].tolist() + ['Challenge Goal Loss'] + new_cols]

    # Save the cleaned data and append to the dataframe list
    data_df.to_csv('./data/cleaned/cleaned_' + data[0] + '.csv')
    
    big_df_list.append(df)

In [32]:
big_df = pd.concat(big_df_list, ignore_index=True).dropna()
big_df.to_csv('./data/cleaned_and_combined_loseit_challenge_data.csv')

Now that we have the data saved and cleaned, we can move onto [Analyze Challenge Data](analyze_loseit_challenge_data.ipynb) to analyze the data and maybe make some predictions.

In [33]:
len(big_df_list)

11

In [34]:
for ex in big_df_list:
    print(len(ex))

0
1022
986
860
889
752
860
800
744
1068
682
